In [1]:
import mido
mido.get_input_names()

['Midi Through:Midi Through Port-0 14:0',
 'Midi Through:Midi Through Port-0 14:0']

In [2]:
midiin_port = 'Midi Through:Midi Through Port-0 14:0'

In [3]:
from sc3.all import *

import ipywidgets as widgets
import numpy as np

from utils import boot, toolbar, button, nodewidgets, guiwidgets, MidiInput

INFO:sc3.synth.server:setting global variable 's' to 'localhost'


In [4]:
boot()

INFO:sc3.synth.server:booting server 'localhost' on address 127.0.0.1:57110
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustJPverbRaw numControls=11
INFO:SERVER.stdout:Found 0 LADSPA plugins
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustGreyholeRaw numControls=7
INFO:SERVER.stdout:JackDriver: client name is 'SuperCollider'
INFO:SERVER.stdout:SC_AudioDriver: sample rate = 48000.000000, driver's block size = 1024
INFO:SERVER.stdout:JackDriver: connected  system:capture_1 to SuperCollider:in_1
INFO:SERVER.stdout:JackDriver: connected  system:capture_2 to SuperCollider:in_2
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_1 to system:playback_1
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_2 to system:playback_2
INFO:SERVER.stdout:SuperCollider 3 server ready.
INFO:SERVER.stdout:JackDriver: max output latency 42.7 ms
INFO:sc3.synth._serverstatus:'localhost': requested re

# FM2 Automata

In [5]:
@synthdef
def fm2(mul1,mul2,add1,add2,amp,lagtime):
    local = LocalIn.ar(2)
    source1 = local[1]*VarLag.kr(mul1,lagtime) + VarLag.kr(add1,lagtime)
    source2 = local[0]*VarLag.kr(mul2,lagtime) + VarLag.kr(add2,lagtime)
    osc1 = SinOsc(source1)
    osc2 = SinOsc(source2)
    out = [osc1,osc2]
    LocalOut(out)    
    Out(0,[o*VarLag.kr(amp,lagtime) for o in out])
main.sync(s);

In [6]:
n = fm2(0,100,5,400,0.8)

In [7]:
tempoclock = TempoClock(2)

In [8]:
midiin = MidiInput(midiin_port)

In [9]:
########### synth parameters control ############

synth_conf = {
 'mul1': {'min':0,'max':600,'step':0.01,'value':1, 'cc':1},
 'mul2': {'min':0,'max':600,'step':0.01,'value':100, 'cc':2},
 'add1': {'min':0,'max':800,'step':0.01,'value':5, 'cc':3},
 'add2': {'min':0,'max':800,'step':0.01,'value':400, 'cc':4},
 'amp': {'min':0,'max':1,'step':0.01,'value':0.8, 'cc':5},
 'lagtime': {'min':-2,'max':1.5,'step':0.01,'value':0.5,'scale':'log', 'cc':6}
}

rhythm_conf = {
 'tempo': {'min':0,'max':16,'step':0.01,'value':8, 'cc':7},
 'rhythm': {'min':0,'max':30,'step':0.01,'value':1, 'cc':8},
 'dmin': {'min':0,'max':2,'step':0.01,'value':0.1, 'cc':9}
}


In [10]:
########### rhythm routine ############

n_steps = widgets.IntSlider(min=1, max=12, step=1, value=4)
steps = []

def randomize_steps(x):
    steps.clear()
    for step in range(n_steps.value):
        d = {}
        for param, conf in synth_conf.items():
            if param not in ['lagtime','amp']:
                d[param] = np.random.uniform(conf['min'],conf['max'])
        steps.append(d)
        
randomize_steps(None)

rhythm_param = {}

@routine
def rhythm_routine():
    d = 0
    step = -1
    while True:
        step += 1
        d = (np.cos(rhythm_param['rhythm']*d)+1) + rhythm_param['dmin']
        if len(steps)>0:
            params = steps[step%len(steps)]
            for w in wf.widget.children:
                state = w.get_state()
                if 'description' in state:
                    key = state['description']
                    if key in params:
                        w.value=params[key]
        yield d
        
def rhythm_func(**kwargs):
    try:
        tempoclock.tempo = kwargs['tempo']
    except:
        pass
    rhythm_param['rhythm'] = kwargs['rhythm']
    rhythm_param['dmin'] = kwargs['dmin']
    
def pause(x):
    if rhythm_routine.state.value==1:
        rhythm_routine.play(tempoclock)  
    elif rhythm_routine.state.value==4:
        rhythm_routine.resume()    
    elif rhythm_routine.state.value in [2,3]:
        rhythm_routine.pause()
    elif rhythm_routine.state.value==5:
        rhythm_routine.reset()
        rhythm_routine.play(tempoclock)

In [1]:
wf = nodewidgets(n, synth_conf, midiin);
guiwidgets(rhythm_conf, rhythm_func, midiin);
        
display(button('Play/Pause',pause))
display(button('Randomize Steps',randomize_steps))
display(n_steps)
display(toolbar())

NameError: name 'nodewidgets' is not defined